In [7]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Assuming all imports are done correctly, let's proceed with the modifications and debugging.

data = pd.read_csv('/content/drive/MyDrive/dielectron.csv')

data.dropna(inplace=True)

print(data.isna().sum())

# Dropping 'Run' and 'Event' columns
data.drop(columns=['Run', 'Event'], inplace=True)

# Separating features and target variable
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Defining Neural Network
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64, 128)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.bn1(x)
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x

# Training Neural Network
input_size = X_train.shape[1]
model = NeuralNet(input_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1)

num_epochs = 30
batch_size = 64

# Splitting the training data further into train and validation sets
X_train_tensor, X_val_tensor, y_train_tensor, y_val_tensor = train_test_split(X_train_tensor, y_train_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

best_val_loss = float('inf')
early_stopping_counter = 0
early_stopping_patience = 5  # Define early stopping patience
for epoch in range(num_epochs):
    # Training
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_inputs, val_targets in val_loader:
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_targets.unsqueeze(1)).item()

    val_loss /= len(val_loader)

    # Learning rate scheduler
    scheduler.step(val_loss)  # Move this line inside the training loop

    # Print training and validation information
    print(f'Epoch [{epoch+1}/{num_epochs}], Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}, '
          f'Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}')

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= early_stopping_patience:
        print(f'Early stopping! No improvement in validation loss for {early_stopping_patience} epochs.')
        break

# Testing
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
test_loss = 0.0
with torch.no_grad():
    for test_inputs, test_targets in test_loader:
        test_outputs = model(test_inputs)
        test_loss += criterion(test_outputs, test_targets.unsqueeze(1)).item()

test_loss /= len(test_loader)

print(f'Test Loss: {test_loss:.4f}')

# Training XGBoost model
xgb_model = XGBRegressor()
xgb_model.fit(X_train_scaled, y_train)

# Predictions using XGBoost
y_pred_xgb = xgb_model.predict(X_test_scaled)

# Calculating mean squared error for XGBoost model
xgb_mse = mean_squared_error(y_test, y_pred_xgb)
print(f'XGBoost Test MSE: {xgb_mse:.4f}')


Run      0
Event    0
E1       0
px1      0
py1      0
pz1      0
pt1      0
eta1     0
phi1     0
Q1       0
E2       0
px2      0
py2      0
pz2      0
pt2      0
eta2     0
phi2     0
Q2       0
M        0
dtype: int64
Epoch [1/30], Learning Rate: 0.010000, Training Loss: 57.5316, Validation Loss: 22.3995
Epoch [2/30], Learning Rate: 0.010000, Training Loss: 21.2112, Validation Loss: 19.3913
Epoch [3/30], Learning Rate: 0.010000, Training Loss: 31.4446, Validation Loss: 10.9130
Epoch [4/30], Learning Rate: 0.010000, Training Loss: 51.7479, Validation Loss: 10.4113
Epoch [5/30], Learning Rate: 0.010000, Training Loss: 150.5732, Validation Loss: 10.3988
Epoch [6/30], Learning Rate: 0.010000, Training Loss: 27.2299, Validation Loss: 7.4708
Epoch [7/30], Learning Rate: 0.010000, Training Loss: 75.2007, Validation Loss: 11.8513
Epoch [8/30], Learning Rate: 0.010000, Training Loss: 8.0120, Validation Loss: 16.1064
Epoch [9/30], Learning Rate: 0.010000, Training Loss: 21.7725, Validation L

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
